In [ ]:
! pip install pyspark

In [8]:
import pyspark
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder.getOrCreate()

In [13]:
produtos = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/produtos.csv', header=True, inferSchema=True)
vendedores = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/vendedores.csv', header=True, inferSchema=True)
clientes = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/clientes.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/itens_pedido.csv', header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/pagamentos_pedido.csv', header=True, inferSchema=True)
avaliacoes_pedido = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/avaliacoes_pedido.csv', header=True, inferSchema=True)
pedidos = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/pedidos.csv', header=True, inferSchema=True)

In [14]:
print("Dataframe Produtos: \n", produtos.show(n=5, truncate=False))
print("Dataframe Vendedores: \n", vendedores.show(5))
print("Dataframe Clientes: \n", clientes.show(5))
print("Dataframe Itens Pedido: \n", itens_pedido.show(5))
print("Dataframe Pagamentos Pedido: \n", pagamentos_pedido.show(5))
print("Dataframe Avaliacoes Pedido: \n", avaliacoes_pedido.show(5))
print("Dataframe Pedidos: \n", pedidos.show(5))

+--------------------------------+---------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|id_produto                      |categoria_produto    |tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------------------+---------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff4541ed26657ea517e5|perfumaria           |40                  |287                      |1                       |225           |16                    |10               |14                |
|3aa071139cb16b67ca9e5dea641aaa2f|artes                |44                  |276                      |1                       |1000          |30                    |18               |

In [15]:
# Acessando colunas
from pyspark.sql.functions import col

clientes.select('id_cliente').show(1)
clientes.select(col('id_cliente')).show(1)
clientes.select(clientes['id_cliente']).show(1)
clientes.select(clientes.id_cliente).show(1)

+--------------------+
|          id_cliente|
+--------------------+
|06b8999e2fba1a1fb...|
+--------------------+
only showing top 1 row

+--------------------+
|          id_cliente|
+--------------------+
|06b8999e2fba1a1fb...|
+--------------------+
only showing top 1 row

+--------------------+
|          id_cliente|
+--------------------+
|06b8999e2fba1a1fb...|
+--------------------+
only showing top 1 row

+--------------------+
|          id_cliente|
+--------------------+
|06b8999e2fba1a1fb...|
+--------------------+
only showing top 1 row



In [18]:
produtos_tratar_categ_nulos = produtos.na.fill({'categoria_produto': 'Não especificado'})
produtos_tratar_categ_nulos.filter(col('categoria_produto')=='Não especificado').count()

610

In [20]:
print('Total de pedidos: ', pedidos.count())

pedidos_unicos = pedidos.dropDuplicates()
print('Total de pedidos únicos: ', pedidos_unicos.count())

pedidos_remocao_nulos = pedidos_unicos.na.drop()
print('Total de pedidos únicos após remoção de nulos: ', pedidos_remocao_nulos.count())

pedidos_remocao_nulos_id = pedidos_unicos.na.drop(subset=['id_cliente', 'id_pedido'])
print('Total de pedidos únicos após limpeza de ids nulos: ', pedidos_remocao_nulos_id.count())

Total de pedidos:  99441
Total de pedidos únicos:  99441
Total de pedidos únicos após remoção de nulos:  96461
Total de pedidos únicos após limpeza de ids nulos:  99441


In [23]:
colunas = ['peso_produto_g', 'comprimento_produto_cm', 'altura_produto_cm', 'largura_produto_cm']

for c in colunas:
  produtos = produtos.na.fill({c: 0})

In [24]:
produtos.write.mode('overwrite').option('header', 'true').csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/output/produtos_tratada_csv')

In [25]:
spark.read.option('header', 'true').csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/output/produtos_tratada_csv').show()

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|
|3aa071139cb16b67c...|               artes|                  44|                      276|                       1|          1000|                    30|               18|                20|
|96bd76ec8810374ed...|       esporte_lazer|  

In [26]:
spark.stop()